In [0]:
%run "../includes/configuration"

In [0]:
race_results_df = spark.read.parquet(f"{presentation_folder_path}/race_results")

In [0]:
display(race_results_df)

In [0]:
from pyspark.sql.functions import sum, count, when, col

constructor_standing_df = race_results_df\
    .groupBy("race_year", "team") \
    .agg(sum("points").alias("total_points"), 
         count(when(col("position") == 1, True)).alias("wins"))

In [0]:
display(constructor_standing_df.filter("race_year = 2020"))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank

constructor_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins"))
final_df = constructor_standing_df.withColumn("rank", rank().over(constructor_rank_spec))



In [0]:
display(final_df.filter("race_year = 2020"))

In [0]:
# final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/constructor_standings")

In [0]:
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.constructor_standings")